In [12]:
import pandas as pd
import os
import sys
import numpy as np
import shutil
import glob

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [13]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '3_fair_data/index_data/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'built_housing_median_age_metric.csv'
Saved DataFrame as 'built_housing_mobile_homes_metric.csv'
Saved DataFrame as 'built_housing_quality_metric.csv'
Saved DataFrame as 'built_metric_housing_vacancy_metric.csv'
Saved DataFrame as 'built_power_plant_metric.csv'
Saved DataFrame as 'climate_wildfire_redbooks_loss_metric.csv'
Saved DataFrame as 'climate_wildfire_redbooks_loss_metric_buildings.csv'
Saved DataFrame as 'climate_wildfire_redbooks_loss_metric_fatalities.csv'
Saved DataFrame as 'governance_emergency_management_metric.csv'
Saved DataFrame as 'governance_hazard_mitigation_metric.csv'
Saved DataFrame as 'natural_air_quality_metric.csv'
Saved DataFrame as 'society_ambulatory_disability_metric.csv'
Saved DataFrame as 'society_american_indian_alaska_native_metric.csv'
Saved DataFrame as 'society_cognitive_disability_metric.csv'
Saved DataFrame as 'society_financial_assistance_metric.csv'
Saved DataFrame as 'society_food_access_metric.csv'
Saved DataFrame as 'societ

In [14]:
# Define the output folder path
output_folder = 'output_folder'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Find all CSV files starting with 'society_'
source_files = [file for file in glob.glob('society_*.csv') if 'social' not in file]

# Iterate through the source files and copy only CSV files to the output folder
for file in source_files:
    # Construct the destination file path
    destination_path = os.path.join(output_folder, os.path.basename(file))
    
    # Copy the file to the output folder
    shutil.copyfile(file, destination_path)
    
    # Remove the original file
    os.remove(file)

print(f"Copied and removed {len(source_files)} CSV files.")

# Delete all CSV files in the current directory that are not in the output folder
current_files = glob.glob('*.csv')
for file in current_files:
    if file not in [os.path.basename(f) for f in source_files]:
        os.remove(file)

print(f"Deleted {len(current_files) - len(source_files)} local CSV files.")


Copied and removed 21 CSV files.
Deleted 1 local CSV files.


In [16]:
# Define the output folder path
output_folder = 'output_folder'

# Get a list of all CSV files in the output folder
csv_files = glob.glob(os.path.join(output_folder, '*.csv'))

# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()
# Iterate through each CSV file and merge them on the 'census_tract' column
for file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Keep only the 'census_tract' and the last column from each file
    last_column = df.columns[-1]
    df = df[['census_tract', last_column]]
    
    # Merge the DataFrame with the existing merged DataFrame
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='census_tract', how='outer')

# Define the path for the final merged CSV file
final_csv_path = os.path.join(output_folder, 'merged_output.csv')

# Save the merged DataFrame to a CSV file
merged_df.to_csv(final_csv_path, index=False)

print(f"Merged CSV saved as {final_csv_path}")

Merged CSV saved as output_folder\merged_output.csv
